# Track Mesoescale Convective Systems (MCS) at consecutive time steps to form tracks with a scheme based on brightness temperature and precipitation

This notebook tracks detected MCS at individual time steps to form tracks. In production, you can use the `scripts/1_detect_and_track_MCS_scheme_Tb_and_P.py` for this step.

The principle for detecting the MCS is from the cold top of the clouds, according to a limiting range of brightness temperature, and an approximate horizontal area generating from the convex hull. The algorithm has the option to operate with only brightness temperature or associating this scheme with precipitation features. The selection criteria and filters for considering or not a MCS can be modified. The parameterization in this notebook was established from an extensive literature review, which can be consulted below.

## The detection of the MCS (regions) is performed using these steps:

1. At any time pixel, find all where brightness temperature `Tb` $\le 225 K$ and trace an approximate region, with the convex hull, according to a binary structure where the pixels that satisfy the described condition are equal to $1$ and those that do not are equal to $0$.
2. Transform from geographic to plane coordinates the pixels and compute an approximate area of those regions traced. 
3. Discard all regions whose area is $< 2000 km^2$.
4. Estimate the average, minimum and maximum brightness temperature of those regions.
5. At any time pixel, find all where precipitation `P` $\ge 2 mm \times {h^{-1}}$ and discard pixels that do not match with the regions estimated in that time step.
6. Estimate an approximate area of those precipitation pixels, that satisfy the previous condition and that are contained in the regions.
7. Estimate the average and maximum precipitation for each region whose area is $\ge 500 km^2$. The algorithm has the option of discard those regions whose precipitaion area is $< 500 km^2$, but in this case those regions are going to be part of the possible tracks.

## The tracks are performed using these steps:

Specifically, assume we have detected $n$ MSC at time $t$, and $m$ MSC at time $t+1$. There are theoretically $n \times m$ possible associations to link these two groups of MCS. Of course not all of them are meaningful. The rules that are applied in the association process are:

1. **overlapping priority** principle: for any MCS at time $t$, the MCS with the highest percentage of overlap at time $t+1$ "wins" and is associated with it. 
2. The MCS with the lowest percentage of overlap at time $t+1$ could form a track on their own, and waits to be associated in the next iteration between $t+1$ and $t+2$.
3. No merging or splitting is allowed, any MCS at time $t$ can only be linked to one MCS at time $t+1$, similarly, any MCS at time $t+1$ can only be linked to one MCS at time $t$.
4. All tracks that do not get updated during the $t$ - $t+1$ process terminate. This assumes that no gap in the track is allowed. 
5. In this first part no tracks are discarded based on their total duration. The algorithm has the option for filtering the tracks with a specific minimun duration or not.

## Input data

**Brightness Temperature:**
NCEP/CPC L3 (Merge IR V1): Spatial and temporal resolution is 4 km and 30 minutes, 
data availability from February 7, 2000 to present. The interest variable of this dataset is `Tb` and the files format must be `netCDF4`.
https://doi.org/10.5067/P4HZB9N27EKU

**Precipitation:**
GPM (IMERG V06B): Spatial and temporal resolution is 10 km and 30 minutes, 
data availability from June 1, 2000 to present. The interest variable of this dataset is `PrecipitationCal` and the files format must be `netCDF4`.
https://doi.org/10.5067/GPM/IMERG/3B-HH/06

In this case the algorithm is run for 5 days (2019/12/25/ 00 (UTC) - 2019/12/31/ 00 (UTC)) for northern South America. The input data are in the notebooks folder in the repository. The raw data can be downloaded with the links at the top.

## Steps

1. Make sure you have successfully installed the ATRACKCS library.
2. Execute the following code blocks in sequence.

## Results

* `resume_Tb_P_2019_12_24_19_2019_12_31_17.csv`: a csv table listing various attributes for the tracks and MCS associated.
* `map_Tb_P_2019_12_24_19_2019_12_31_10.html` (folium): plot of the geographical locations of the MSC with informations that links to the associaded tracks and features of the MSC.

## Bibliography
* Feng, Z., Leung, L. R., Liu, N., Wang, J., Houze, R. A., Li, J., Hardin, J. C., Chen, D., & Guo, J. (2021). A Global High‐resolution Mesoscale Convective System Database using Satellite‐derived Cloud Tops, Surface Precipitation, and Tracking. Journal of Geophysical Research: Atmospheres. https://doi.org/10.1029/2020jd034202
* Li, J., Feng, Z., Qian, Y., & Leung, L. R. (2020). A high-resolution unified observational data product of mesoscale convective systems and isolated deep convection in the United States for 2004–2017. Earth System Science Data Discussions, October, 1–48. https://doi.org/10.5194/essd-2020-151
* Liu, W., Cook, K. H., & Vizy, E. K. (2019). The role of mesoscale convective systems in the diurnal cycle of rainfall and its seasonality over sub-Saharan Northern Africa. Climate Dynamics, 52(1–2), 729–745. https://doi.org/10.1007/s00382-018-4162-y
* Vizy, E. K., & Cook, K. H. (2018). Mesoscale convective systems and nocturnal rainfall over the West African Sahel: role of the Inter-tropical front. Climate Dynamics, 50(1–2), 587–614. https://doi.org/10.1007/s00382-017-3628-7

### Set paths

First of all we assign the locations to the input (raw data) using `TBDIR` and `PDIR` and output data using `OUTDIR`.

In [1]:
%matplotlib inline
import os

TBDIR=os.path.join('FAC', 'tb/')
PDIR=os.path.join('FAC', 'p/')

OUTDIR=os.path.join('FAC, output_data/')

### Parameters used in the MCS detection and tracking process

* `UTM_LOCAL_ZONE`: int, is needed for converting the WGS geodetic coordinate system to plane coordinate system. This is a constant that must be asociated with the interest region. 
* `UTC_LOCAL_HOUR`: int, is needed for converting the raw data hour (UTC) to a local hour (interest region).
* `UTC_LOCAL_SIGN`: str (minus, plus, local), is needed for converting the raw data hour (UTC) to a local hour (interest region).
* `DETECT_SCHEME`: str (Tb, Both), association scheme: Tb or Both (Tb and P). 
* `TB`: int, MCS based on limited maximun threshold cold cloud top.
* `AREA_TB`: int, MCS with a minimun largest area.
* `MIN_P`: int, MCS with a minimun precipitation pixel.
* `AREA_P`: int, MCS with an mimimum area precipitation.
* `DROP_EMPTY_PRECIPITATION`: boolean, if `True` eliminates MCS that do not contain precipitation with the `MIN_P` and `AREA_P` selected. 
* `THRESHOLD_OVERLAPPING_P`: int, percentage overlap limit between MCS.
* `LOCATION_FOLIUM`: list (lat, lon), location for center the map_folium.
* `MIN_DURATION`: int, minimum required number of hours of a track.


In [2]:
#32718 is the UTM zone 18S plane coordinate system. 
#It was Used for tracking MCS in South America - Colombia.
UTM_LOCAL_ZONE = 32718 

#UTC-5 is the local hour for Colombia
UTC_LOCAL_HOUR = 0
UTC_LOCAL_SIGN = "local"

#Scheme of association
DETECT_SCHEME = "Both"

#RANGE Tb 
#Tb [200 k - 240 k]
#RANGE AREA TB
#area_Tb[>999 km]

TB = 225 #[Feng et al.,(2021); Li et al.,(2020)]

AREA_TB = 2000 # [Lui et al., (2019); Vizy & Cook,(2018)] 

#NO RESTRICTIONS AREA_P AND MIN_P

MIN_P = 2 #[Feng et al.,(2021)]  

AREA_P = 500 #[Feng et al.,(2021)]

DROP_EMPTY_PRECIPITATION = False

THRESHOLD_OVERLAPPING_P = 0

LOCATION_FOLIUM = [5, -73.94]

MIN_DURATION = 6

### Importing modules

In [3]:
#--------Import modules-------------------------
from atrackcs.utils import funcs
from atrackcs.detect import detect_mcs
from atrackcs.features import features_Tb, features_P, features_Tracks
from atrackcs.track import track_mcs

### Reading raw Tb and P data.

The reading process is handled with the function `funcs.readNC()`.

Also make sure the output folder exists.

In [4]:
ds = funcs.readNC(pathTb = TBDIR, pathP =PDIR, utc_local_hour = UTC_LOCAL_HOUR, 
                  utc_local_sign = UTC_LOCAL_SIGN)

if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)

Complete Tb and P data reading 2021-08-01T00:00 - 2021-10-29T23:00


The detection process is handled with the function `detect_mcs()`.

* `ds` is the `xarray.Dataset` object we just read in.

### Detecting MCS

In [5]:
mcs_l = detect_mcs(ds, detect_scheme = DETECT_SCHEME, Tb = TB, area_Tb = AREA_TB, 
                   utm_local_zone = UTM_LOCAL_ZONE, path_save = None)

MCS detection completed


The MCS Tb features is handled with the function `features_Tb()`.

* `mcs_l` is the `Geopandas.GeoDataFrame` object we just created in the detection process.

In [6]:
mcs_l = features_Tb(mcs_l, ds)

Estimating MCS's brightness temperature attributes: 


100%|██████████████████████████████████████████████████████████████████████████████| 8446/8446 [10:06<00:00, 13.93it/s]


* `mcs_l` is the `geopandas.GeoDataFrame` object we just created in the tracking process.

Each `row` object in `mcs_l` stores a MCS record. We can have a peak into what `msc_l` columns contains so far. 

* `time`: datetime64, hour (UTC-5) for this case.
* `Tb`: float, polygon index after being filtered by the established parameterization.
* `geometry`: geometry, MCS polygon (convex hull). To view the coordinate reference system of the geometry column, access the crs attribute. For this case is `EPSG:32718`.
* `area_Tb`: float, area polygon [$km^2$]
* `centroid_`: geometry, geometric centroid polygon (convex hull). The crs in  this case is `EPSG:32718`.
* `mean_tb`: float, brightness temperature average of the pixels composing the polygon. [$K$]
* `min_tb`: float, brightness temperature min value of the pixels composing the polygon. [$K$]
* `max_tb`: float, brightness temperature max value of the pixels composing the polygon. [$K$]

For this case 3201 MCS were detected with the selected parameterization.

In [7]:
mcs_l

,time,Tb,geometry,area_tb,centroid_,mean_tb,min_tb,max_tb
0,2021-08-01 00:00:00,8.0,"POLYGON ((294642.098 10458947.737, 283565.248 ...",2148.3,POINT (287338.854 10498042.790),219.9816,214.6666,225.6053
1,2021-08-01 00:00:00,9.0,"POLYGON ((328194.345 10569453.757, 294964.065 ...",3370.0,POINT (308289.087 10610189.288),216.1496,207.0629,224.4173
2,2021-08-01 00:00:00,10.0,"POLYGON ((1664193.249 10601335.998, 1641481.25...",7907.5,POINT (1662321.536 10648509.021),219.3849,212.1818,228.1542
3,2021-08-01 00:00:00,11.0,"POLYGON ((-1071492.357 10643778.552, -1116882....",4688.8,POINT (-1097611.089 10682158.931),221.5973,212.9739,236.5700
4,2021-08-01 00:00:00,12.0,"POLYGON ((1911953.809 10651380.608, 1866366.19...",16445.4,POINT (1867410.108 10731151.235),222.9712,202.7540,267.0389
...,...,...,...,...,...,...,...,...
8442,2021-10-29 23:00:00,177.0,"POLYGON ((-978464.580 11539489.685, -988914.46...",16443.6,POINT (-913169.283 11585989.119),219.2127,195.8632,283.3979
8443,2021-10-29 23:00:00,178.0,"POLYGON ((-548108.484 11529993.541, -546745.39...",8367.5,POINT (-460348.267 11580708.881),222.9454,210.0641,242.7126
8444,2021-10-29 23:00:00,179.0,"POLYGON ((-807228.270 11632171.059, -817612.40...",12950.6,POINT (-706214.722 11665369.148),219.3431,196.3320,273.6818
8445,2021-10-29 23:00:00,181.0,"POLYGON ((-814509.498 11700446.829, -824834.46...",30944.8,POINT (-757692.890 11807364.440),214.1800,198.4479,241.3825


In [8]:
mcs_l.crs

<Projected CRS: EPSG:32718>
Name: WGS 84 / UTM zone 18S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 78°W and 72°W, southern hemisphere between 80°S and equator, onshore and offshore. Argentina. Brazil. Chile. Colombia. Ecuador. Peru.
- bounds: (-78.0, -80.0, -72.0, 0.0)
Coordinate Operation:
- name: UTM zone 18S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

The MCS P features is handled with the function `features_P()`.

In [9]:
mcs_l = features_P(mcs_l, ds, min_precipitation = MIN_P, area_P = AREA_P,
                   drop_empty_precipitation = DROP_EMPTY_PRECIPITATION)

Estimating MCS's precipitation attributes: 


100%|██████████████████████████████████████████████████████████████████████████████| 8446/8446 [10:03<00:00, 13.99it/s]


### Tracking MCS

The tracking process is handled with the function `track_mcs()`. However, these have not been characterized in this phase. The tracks features are handled with the function `features_Tracks()`.

In [10]:
tracks_l = track_mcs(mcs_l, threshold_overlapping_percentage = THRESHOLD_OVERLAPPING_P, utm_local_zone = UTM_LOCAL_ZONE,
                path_save = None)

Estimating trajectories: 


100%|██████████████████████████████████████████████████████████████████████████████| 8446/8446 [14:26<00:00,  9.75it/s]


In [11]:
tracks_l = features_Tracks(tracks_l, initial_time_hour = MIN_DURATION,
                         path_save = OUTDIR)

Estimating distance and direction between geometrics centroids: 


100%|██████████████████████████████████████████████████████████████████████████████| 3105/3105 [01:41<00:00, 30.66it/s]


* `tracks_l` is a `geopandas.GeoDataFrame` object we just created in the tracking process and stores the MCS polygons and the tracks. 
* This `GeoDataFrame` contains exactly the information previously referenced except for the dropping of some features associated to the brightness temperature and the addition of some characteristics associated to the MCS precipitation and some characteristics generated for the tracks.
* The indexing of this object is a `pandas.MultiIndex` since it hierarchically associates an id for each track and an id for each MCS.
* The indexing of this object is encrypted generated using the `uuid` library. To disable encryption for both MCS and tracks, use the parameter `encrypt_index = False` in the `features_Tracks()` function. The index generated when not using encryption is a `int` iteration result  of the tracking process.
* The indexes encryption is useful when processing a long period of time and must iterate for smaller periods of time, for example every 6 months. This is a limitation imposed by the hardware used to run the algorithm.

We can have a peak into what `tracks_l` new columns contains. 


* `belong`: str, encrypted index generated for each track.
* `id_gdf`: str, encrypted index generated for each MCS.
* `geometry`: geometry, polygon. The crs is `EPSG:4326-WGS84`
* `centroid_`: geometry, geometric centroid polygon. The crs is `EPSG:4326`.
* `mean_pp`: float, precipitation average of the pixels composing the polygon [$mm \times {h^{-1}}$].
* `max_pp`: float, precipitation max value of the pixels composing the polygon [$mm \times {h^{-1}}$].
* `intersection_percentage`: float, percentage overlap between MCS [%].
* `distance_c`: float, distance between the overlapping geometric centroids of the MCS [km].
* `direction`: float, direction between the overlapping geometric centroids of the MCS [°].
* `total_duration`: float, total duration of the event or the track. This value is associated to each MCS of the corresponding track [h]. 
* `total_distance`: float, total distance of the event or the track. This value is associated to each MCS of the corresponding track [km]. 
* `mean_velocity`: float, velocity average of the  event or the track. This value is associated to each MCS of the corresponding track [$km \times {h^{-1}}$]. 

In [12]:
tracks_l

time  \
belong        id_gdf                                       
00187b65-15ea 4397-abbb-3555b04fb8ae 2021-09-24 20:00:00   
              48ff-9e12-3c8d4588c179 2021-09-25 00:00:00   
              4964-9d46-61e79179c644 2021-09-24 19:00:00   
              4d92-98aa-e9dda63dec77 2021-09-24 22:00:00   
              4e76-acd2-e2212b8fa9e1 2021-09-24 23:00:00   
...                                                  ...   
fc0d2a47-7012 4ab3-84ca-9f038c48d9e7 2021-09-22 07:00:00   
              4b37-9bca-6c488e948c1c 2021-09-22 08:00:00   
              4cc3-9024-4741c7a5acce 2021-09-22 00:00:00   
              4d50-a634-06cebb75a3fb 2021-09-22 06:00:00   
              4f79-b45d-6ff2697f9e1d 2021-09-22 05:00:00   

                                                                               geometry  \
belong        id_gdf                                                                      
00187b65-15ea 4397-abbb-3555b04fb8ae  POLYGON ((-73.85000 0.45000, -73.95000 0.55000...   
              48ff-9e12-3c8d4588c179  POLYGON ((-74.65000 1.65000, -74.65000 2.15000...   
              4964-9d46-61e79179c644  POLYGON ((-72.75000 1.55000, -72.95000 1.95000...   
              4d92-98aa-e9dda63dec77  POLYGON ((-74.25000 0.75000, -74.35000 0.85000...   
              4e76-acd2-e2212b8fa9e1  POLYGON ((-74.45000 0.75000, -74.55000 0.85000...   
...                                                                                 ...   
fc0d2a47-7012 4ab3-84ca-9f038c48d9e7  POLYGON ((-68.45000 2.75000, -69.05000 2.95000...   
              4b37-9bca-6c488e948c1c  POLYGON ((-69.35000 2.85000, -69.45000 3.05000...   
              4cc3-9024-4741c7a5acce  POLYGON ((-67.85000 2.85000, -67.95000 2.95000...   
              4d50-a634-06cebb75a3fb  POLYGON ((-68.05000 2.85000, -68.45000 2.95000...   
              4f79-b45d-6ff2697f9e1d  POLYGON ((-67.45000 2.65000, -68.35000 2.85000...   

                                      area_tb                  centroid_  \
belong        id_gdf                                                       
00187b65-15ea 4397-abbb-3555b04fb8ae  14953.4  POINT (-73.35840 1.25312)   
              48ff-9e12-3c8d4588c179   3195.8  POINT (-74.44490 2.13589)   
              4964-9d46-61e79179c644   2585.6  POINT (-72.69919 1.86268)   
              4d92-98aa-e9dda63dec77  17219.6  POINT (-73.98021 1.45800)   
              4e76-acd2-e2212b8fa9e1  12665.6  POINT (-74.23255 1.59287)   
...                                       ...                        ...   
fc0d2a47-7012 4ab3-84ca-9f038c48d9e7  44093.2  POINT (-67.66171 3.76872)   
              4b37-9bca-6c488e948c1c  50420.5  POINT (-67.87038 3.83996)   
              4cc3-9024-4741c7a5acce   2495.7  POINT (-67.79077 3.11581)   
              4d50-a634-06cebb75a3fb  21068.6  POINT (-67.93402 3.49872)   
              4f79-b45d-6ff2697f9e1d  23529.9  POINT (-67.68301 3.31131)   

                                       mean_tb  mean_pp   max_pp  \
belong        id_gdf                                               
00187b65-15ea 4397-abbb-3555b04fb8ae  222.5814   4.3826   9.4014   
              48ff-9e12-3c8d4588c179  224.3726      NaN      NaN   
              4964-9d46-61e79179c644  217.7535      NaN      NaN   
              4d92-98aa-e9dda63dec77  218.7721   3.5859  19.7889   
              4e76-acd2-e2212b8fa9e1  223.1849   2.6519   4.3409   
...                                        ...      ...      ...   
fc0d2a47-7012 4ab3-84ca-9f038c48d9e7  218.0966   4.6533   9.9393   
              4b37-9bca-6c488e948c1c  221.7950   7.1384  33.2969   
              4cc3-9024-4741c7a5acce  205.1374   4.3045   6.3069   
              4d50-a634-06cebb75a3fb  211.0295   7.3695  14.7163   
              4f79-b45d-6ff2697f9e1d  215.5185   8.8993  16.9644   

                                      intersection_percentage  distance_c  \
belong        id_gdf                                                        
00187b65-15ea 4397-abbb-3555b04fb8ae                     48

In [13]:
tracks_l.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
#Total tracks and MCS detected 

print ("total tracks detected: " + str(len(tracks_l.index.levels[0])) + " and total MCS detected: " + str(len(tracks_l.index.levels[1])))

total tracks detected: 276 and total MCS detected: 2104


### As the last step, let is reload the tracks results from the local storage and plot the MCS with the help of the folium library.

The read process tracks is handled with the function `funcs.readTRACKS()`.


In [27]:
tracks_l = funcs.readTRACKS('FAC_output_data/resume_Tb_P_2021_08_01_00_2021_10_29_23.csv')

#---------Discarding tracks that last less than 10 hours--------
tracks_l = tracks_l.reset_index()
tracks_l = tracks_l.set_index("time")

In [32]:
#-------------------Load results-------------------
tracks_l = funcs.readTRACKS('FAC_output_data/resume_Tb_P_2021_08_01_00_2021_10_29_23.csv')

#---------Discarding tracks that last less than 10 hours--------
tracks_l = tracks_l.reset_index()
tracks_l = tracks_l.set_index("time")

for mes in ["2021/08","2021/09","2021/10"]:
    tracks_ = tracks_l[mes]
    tracks_ = tracks_.reset_index()
    tracks_ = tracks_.set_index(["belong", "id_gdf"])
    tracks_ = tracks_.sort_index()
    path_html_folium = funcs.plot_folium(tracks_, location = LOCATION_FOLIUM, path_save = OUTDIR)
    
#print ("total tracks that last less than 10 hours : " + str(len(tracks_l.index.levels[0])) + " and total MCS detected: " + str(len(tracks_l.index.levels[1])))

The function `funcs.plot_folium()` saves the `.html` result and return the path where was saved.


In [16]:
path_html_folium = funcs.plot_folium(tracks_l, location = LOCATION_FOLIUM, path_save = OUTDIR)

In [17]:
import IPython
iframe = '<iframe src=' + path_html_folium + ' width=1000 height=500></iframe>'
IPython.display.HTML(iframe)